In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# columns of interest
cols_to_extract = ['eventid','imonth', 'iyear','iday',
 'country_txt','region_txt','provstate','city','latitude','longitude',
 'crit1','crit2','crit3',
 'attacktype1_txt',
 'success','suicide',
 'weaptype1_txt',
 'targtype1_txt',
 'gname',
 'individual','nperps','nperpcap',
 'claimed',
 'nkill','nkillter','nwound','nwoundte',
 'property','propextent_txt',
 'ishostkid','nhostkid','nhours','ndays','kidhijcountry',
 'ransom','ransomamt','ransompaid','hostkidoutcome_txt','nreleased']

# Load data
df = pd.read_excel('GTD_0617dist/globalterrorismdb_0617dist.xlsx',
                   index_col=0,usecols=cols_to_extract,
                   na_values = ['Unknown','-99','-9','Not Applicable'])

# replace some values not correctly dealt with by pandas import
df.replace(-9,np.nan,inplace=True)
df.replace(-99,np.nan,inplace=True)

# create a date column, then get rid of the seaparte year, month and day columns
# entries without month or day - treat as 1st January
df['imonth'].replace(0,1,inplace=True)
df['iday'].replace(0,1,inplace=True)
df['iyear']=pd.to_datetime(dict(year=df.iyear, month=df.imonth, day=df.iday)) 
df.rename(columns={'iyear':'date'}, inplace=True)
df.drop(['imonth','iday'],axis=1)

df.info()

ValueError: 'idaycountry_txt' is not in list

In [ ]:
# look at the most common values in each column
print('TOP 2 VALUES IN EACH COLUMN:')
print('===================================')
for column in df.columns:
    top2 = df[column].value_counts().index[:2]
    print('{}: {}, {}'.format(column,top2[0],top2[1]))

In [ ]:
# Some rows are identical to each other.
# E.g. 100 duplicates of GTD ID 201609210029. In this case seems to be
# 100 separte incidents of an explosive being detonated in a civilian
# house. So I will treat the duplicates as genuine.
dups = df.duplicated(keep=False)
print('There are',dups.sum(),'rows which have duplicated information in other rows.')

In [ ]:
# Plot some basic stats

# total incidents per year
tot_inc_yr = df.resample('A',on='date')['date'].count()
tot_inc_yr.index = tot_inc_yr.index.year
tot_inc_yr.name = 'n_incidents'
tot_inc_yr.index.name = 'year'
plt.figure(figsize=(15,5))
tot_inc_yr.plot.bar(title='No. Global Terrorism Incidents Per Year')
plt.ylabel('No. Incidents');

# total deaths and wounded per year
tot_kill_yr = df.resample('A',on='date')[['nkill','nwound']].sum()
tot_kill_yr.name = 'killed_wounded'
tot_kill_yr.index = tot_kill_yr.index.year
tot_kill_yr.index.name = 'year'

ax=plt.figure(figsize=(15,5)).gca()
tot_kill_yr.plot(kind='bar',stacked=True,ax=ax,title='Killed and Wounded Per Year')
plt.legend();
plt.ylabel('Total');

In [ ]:
# No. of incidents per region
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
df['region_txt'].value_counts().plot.bar(title='Incidents per Region')
plt.ylabel('No. Incidents')

plt.subplot(1,2,2)
df['country_txt'].value_counts()[:20].plot.bar(title='Top 20 Countries with Most Incidents')
plt.ylabel('No. Incidents');

In [ ]:
# MOST ACTIVE GROUPS PER YEAR

# incidents per year per group
grp_inc_yr = df.groupby([pd.Grouper(freq='A',key='date'),'gname'])['country_txt'].count()
grp_inc_yr = grp_inc_yr.unstack('gname',fill_value=0)
grp_inc_yr.name = 'n_incidents'
grp_inc_yr.index = grp_inc_yr.index.year
grp_inc_yr.index.name = 'year'

# fraction incidents per year
grp_frinc_yr = grp_inc_yr.div(tot_inc_yr,axis='rows')
grp_frinc_yr.name = 'frac_incidents'

# heatmap plot function
def plot_heatmap(data,year_range,n_groups=20):
    plot_groups = data.loc[year_range].sum().sort_values(ascending=False)[:n_groups].index
    plt.figure(figsize=(15,5))
    sns.heatmap(data[plot_groups].loc[year_range].T)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Heatmap of {} by Top {} Groups: {} - {}'.format(data.name,n_groups,min(year_range),max(year_range)))

plot_heatmap(grp_frinc_yr,list(np.arange(1970,1993)))
plot_heatmap(grp_frinc_yr,list(np.arange(1994,2017)))

In [ ]:
grp_att=df.groupby('gname')['attacktype1_txt'].value_counts().unstack('attacktype1_txt',fill_value=0)
grp_att['tot_inc'] = grp_att.sum(axis=1)
grp_att.sort_values(by='tot_inc',ascending=False).drop('tot_inc',axis=1).head(20).plot(kind='bar',stacked=True)